In [ ]:
# Jupyter Notebook Template for ISL Translation
# File: training/notebook.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Indian Sign Language Translation\n",
    "## End-to-End Training and Inference Demo"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import os\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import display, HTML\n",
    "from app.processing.landmark_extractor import MediaPipeLandmarkExtractor\n",
    "from training.dataset import ISLDataset\n",
    "from app.models.isl_model import SignLanguageTransformer\n",
    "from app.utils.translation_mapper import TranslationMapper\n",
    "from training.train import train_model\n",
    "from training.evaluate import evaluate_model\n",
    "from app.core.config import load_config"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configuration\n",
    "config = load_config()\n",
    "extractor = MediaPipeLandmarkExtractor(use_gpu=True)\n",
    "translator = TranslationMapper(config.label_mappings)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load sample video\n",
    "video_path = \"data/samples/hello.mp4\"\n",
    "cap = cv2.VideoCapture(video_path)\n",
    "frames = []\n",
    "landmarks_seq = []\n",
    "\n",
    "while cap.isOpened():\n",
    "    ret, frame = cap.read()\n",
    "    if not ret:\n",
    "        break\n",
    "    \n",
    "    # Process frame\n",
    "    landmarks = extractor.process(frame)\n",
    "    landmarks_seq.append(landmarks)\n",
    "    frames.append(frame)\n",
    "\n",
    "cap.release()\n",
    "print(f\"Processed {len(frames)} frames\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize landmarks\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.plot(np.array(landmarks_seq)[:, :21*3])  # Plot hand landmarks\n",
    "plt.title(\"Hand Landmark Trajectories\")\n",
    "plt.xlabel(\"Frame\")\n",
    "plt.ylabel(\"Landmark Position\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare dataset\n",
    "dataset = ISLDataset(\"data/train\", transform=None)\n",
    "print(f\"Dataset size: {len(dataset)}\")\n",
    "print(f\"Input shape: {dataset[0][0].shape}\")\n",
    "\n",
    "# Show class distribution\n",
    "class_counts = {}\n",
    "for _, label in dataset:\n",
    "    class_counts[label.item()] = class_counts.get(label.item(), 0) + 1\n",
    "    \n",
    "plt.bar(class_counts.keys(), class_counts.values())\n",
    "plt.title(\"Class Distribution\")\n",
    "plt.xlabel(\"Class ID\")\n",
    "plt.ylabel(\"Count\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train model\n",
    "model, history = train_model(\n",
    "    train_data_dir=\"data/train\",\n",
    "    val_data_dir=\"data/val\",\n",
    "    epochs=50,\n",
    "    batch_size=32,\n",
    "    learning_rate=0.001\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot training history\n",
    "plt.figure(figsize=(12, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history['train_loss'], label='Train Loss')\n",
    "plt.plot(history['val_loss'], label='Validation Loss')\n",
    "plt.legend()\n",
    "plt.title(\"Loss Curves\")\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history['train_acc'], label='Train Accuracy')\n",
    "plt.plot(history['val_acc'], label='Validation Accuracy')\n",
    "plt.legend()\n",
    "plt.title(\"Accuracy Curves\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Run inference\n",
    "input_sequence = np.array(landmarks_seq[-config.sequence_length:])\n",
    "input_tensor = torch.tensor(input_sequence).unsqueeze(0).float()\n",
    "\n",
    "with torch.no_grad():\n",
    "    output = model(input_tensor)\n",
    "    pred_idx = output.argmax().item()\n",
    "    confidence = output[0][pred_idx].item()\n",
    "\n",
    "print(f\"Prediction: {translator.get_english(pred_idx)} (Hindi: {translator.get_hindi(pred_idx)})\")\n",
    "print(f\"Confidence: {confidence:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create video with prediction overlay\n",
    "output_frames = []\n",
    "for frame in frames[-30:]:  # Last second of video\n",
    "    display_frame = frame.copy()\n",
    "    cv2.putText(display_frame, f\"Prediction: {translator.get_english(pred_idx)}\", \n",
    "                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)\n",
    "    cv2.putText(display_frame, f\"Confidence: {confidence:.2f}\", \n",
    "                (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)\n",
    "    output_frames.append(display_frame)\n",
    "\n",
    "# Save and display result\n",
    "output_path = \"results/output.mp4\"\n",
    "frames_to_video(output_frames, output_path, fps=30)\n",
    "\n",
    "# Display in notebook\n",
    "display(HTML(f\"\"\"\n",
    "<video width=\"640\" height=\"480\" controls>\n",
    "  <source src=\"{output_path}\" type=\"video/mp4\">\n",
    "</video>\n",
    "\"\"\"))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.18"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}